# Passing TDs - Draftkings

This is the url from the network tab: https://sportsbook-nash.draftkings.com/api/sportscontent/dkuswatl/v1/leagues/88808/categories/1000/subcategories/15987

Lets try to access it

In [30]:
import requests

# The URL of the API endpoint
url = "https://sportsbook-nash.draftkings.com/api/sportscontent/dkuswatl/v1/leagues/88808/categories/1000/subcategories/15987"

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9,ja;q=0.8,es;q=0.7",
    "Origin": "https://sportsbook.draftkings.com",
    "Referer": "https://sportsbook-nash.draftkings.com/",
    "Sec-Ch-Ua": '"Chromium";v="130", "Google Chrome";v="130", "Not:A_Brand";v="99"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "Windows",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
}


# Send a GET request to the URL
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse JSON data
    data = response.json()
    print(data)  # Display the JSON data
else:
    print(f"Failed to retrieve data: {response.status_code}")


{'sports': [{'id': '3', 'seoIdentifier': 'Football', 'name': 'Football', 'sortOrder': 58}], 'leagues': [{'id': '88808', 'name': 'NFL', 'sportId': '3', 'sortOrder': 4847, 'tags': ['FeaturedSubcategory', 'MatchTracker', 'ProgressiveParlayTeamEligible', 'SGP', 'Scoreboard', 'SgpDynamicOdds', 'Stats', 'YourBetEligible', 'LiveStreaming', 'PrePack', 'ProgressiveParlayEligible', 'Featured', 'FavoriteableFromPage'], 'isTeamSwap': True, 'featured': True, 'media': [{'providerName': 'BetRadarStats', 'mediaType': 'Stats'}, {'providerName': 'BetRadarV3', 'mediaType': 'MatchTracker'}, {'providerName': 'BetRadarV3', 'mediaType': 'Scoreboard'}, {'providerName': 'BetGenius', 'mediaType': 'LiveStreaming', 'allowedDmas': ['800', '564', '868', '510', '535', '542', '866', '558', '803', '828', '804', '597', '825', '807', '855', '547', '554', '536', '662', '525', '532', '790', '644', '583', '634', '524', '520', '635', '512', '537', '716', '692', '821', '756', '746', '502', '630', '559', '757', '506', '736', 

It works! This is passing TDs. Lets try predict Joe Burrow's passing TDs (no devigging)

First lets get the odds for all of Burrow's passing TDs

In [31]:
def parse_qb_passing_tds_json(name: str):
  selections = data["selections"]
  odds = {}
  for s in selections:
    participants = s["participants"]
    for p in participants:
      if p["name"] == name:
        label = s["label"]
        odds[label] = s["displayOdds"]["american"]

  return odds

In [32]:
burrow_odds = parse_qb_passing_tds_json("Joe Burrow")
burrow_odds

{'1+': '−1100', '2+': '−170', '3+': '+220', '4+': '+850'}

Great! Now we can convert these to implied probabilities

In [33]:
def convert_to_implied_probability(odds):
  positive = odds[0] == '+'
  odds = int(odds[1:])
  if positive:
    return 100 / (odds + 100)
  else:
    return odds / (odds + 100)

In [34]:
def convert_odds_dict_to_implied_probability(oddsDict, atLeast: int = 0):
  implied_probabilities = {tds: convert_to_implied_probability(odds) for tds, odds in oddsDict.items()}
  implied_probabilities[str(atLeast) + '+'] = 1.00 # 100% chance to for at least 0 TDs
  return implied_probabilities

In [35]:
burrow_probabilities = convert_odds_dict_to_implied_probability(burrow_odds)
burrow_probabilities

{'1+': 0.9166666666666666,
 '2+': 0.6296296296296297,
 '3+': 0.3125,
 '4+': 0.10526315789473684,
 '0+': 1.0}

From here, we can calculate the probabilities that Burrow gets exactly 0, 1, 2, 3, and 4+ TDs

In [36]:
def convert_cumulative_to_exact(cumulative_probabilities):
  cumulative_probabilities = { int(k[0]):v for k, v in cumulative_probabilities.items() }
  exact_probabilities = {}
  upper_td_range = max(cumulative_probabilities) # some have 4+ td props, some have only 3
  for i in range(0, upper_td_range):
    exact_probabilities[i] = cumulative_probabilities[i] - cumulative_probabilities[i + 1]

  exact_probabilities[upper_td_range] = cumulative_probabilities[upper_td_range]
  return exact_probabilities

In [37]:
exact_burrow_td_probabilities = convert_cumulative_to_exact(burrow_probabilities)
exact_burrow_td_probabilities

{0: 0.08333333333333337,
 1: 0.287037037037037,
 2: 0.31712962962962965,
 3: 0.20723684210526316,
 4: 0.10526315789473684}

Finally, we can find the expected number of passing TDs by multiplying to find the EV

In [38]:
def find_expected_value(exact_probabilities):
  ev = 0
  for k,v in exact_probabilities.items():
    ev += (k * v)

  return ev

In [39]:
burrow_expected_tds = find_expected_value(exact_burrow_td_probabilities)
burrow_expected_tds

1.964059454191033

We can also calculate the amount of expected fantasy points!

In [40]:
def find_expected_fantasy_points(statistic_value, fantasy_weight):
  return statistic_value * fantasy_weight

In [41]:
find_expected_fantasy_points(burrow_expected_tds, fantasy_weight=4)

7.856237816764132

We expect Joe Burrow to get 7.88 fantasy points based on just passing TDs

Lets make this whole process into one simplified function (and try it out!)

In [42]:
def find_expected_passing_tds(name: str):
  qb_passing_td_odds = parse_qb_passing_tds_json(name)
  implied_probabilities = convert_odds_dict_to_implied_probability(qb_passing_td_odds)
  exact_qb_td_probabilities = convert_cumulative_to_exact(implied_probabilities)
  return find_expected_value(exact_qb_td_probabilities)

In [43]:
find_expected_passing_tds("Joe Burrow")

1.964059454191033

Lets do this for every QB that has props!

In [44]:
def get_all_expected_passing_tds():
  expected_tds = {}
  for s in data["selections"]:
    participants = s["participants"]
    for p in participants:
      if p["name"] not in expected_tds:
        expected_tds[p["name"]] = find_expected_passing_tds(p["name"])

  return expected_tds

In [45]:
get_all_expected_passing_tds()

{'Joe Burrow': 1.964059454191033,
 'Kirk Cousins': 1.9047403576815343,
 'Dak Prescott': 1.8644359664480388,
 'Josh Allen': 1.7230972337355315,
 'Patrick Mahomes': 1.6914727715052087,
 'Sam Darnold': 1.7869086601679558,
 'Matthew Stafford': 1.6126984126984127,
 'Geno Smith': 1.5339587242026267,
 'Jared Goff': 1.5461538461538462,
 'Derek Carr': 1.5461538461538462,
 'Lamar Jackson (BAL)': 1.5461538461538462,
 'Jayden Daniels': 1.5583489681050657,
 'Jalen Hurts': 1.4913419913419914,
 'Justin Herbert': 1.4371980676328504,
 'Joe Flacco': 1.4393939393939394,
 'Tua Tagovailoa': 1.4393939393939394,
 'Jameis Winston': 1.388628762541806,
 'Baker Mayfield': 1.3793780687397708,
 'Caleb Williams': 1.3664311878597593,
 'Kyler Murray': 1.3080988917306051,
 'Trevor Lawrence': 1.256910311294329,
 'Gardner Minshew': 1.1997300944669367,
 'Bo Nix': 1.2059957586273375,
 'Daniel Jones': 1.0486080424215642,
 'Bryce Young': 0.887218045112782}

Find the expected fantasy points from passing TDs too:

In [46]:
def get_all_expected_passing_td_fantasy_points():
  expected_tds = get_all_expected_passing_tds()
  expected_fantasy_points = { player:find_expected_fantasy_points(tds, fantasy_weight=4) for player, tds in expected_tds.items() }
  return expected_fantasy_points

In [47]:
fantasy_points_passing_tds = get_all_expected_passing_td_fantasy_points()
sorted(fantasy_points_passing_tds.items(), key=lambda x: x[1], reverse=True)

[('Joe Burrow', 7.856237816764132),
 ('Kirk Cousins', 7.618961430726137),
 ('Dak Prescott', 7.457743865792155),
 ('Sam Darnold', 7.147634640671823),
 ('Josh Allen', 6.892388934942126),
 ('Patrick Mahomes', 6.765891086020835),
 ('Matthew Stafford', 6.450793650793651),
 ('Jayden Daniels', 6.233395872420263),
 ('Jared Goff', 6.184615384615385),
 ('Derek Carr', 6.184615384615385),
 ('Lamar Jackson (BAL)', 6.184615384615385),
 ('Geno Smith', 6.135834896810507),
 ('Jalen Hurts', 5.965367965367966),
 ('Joe Flacco', 5.757575757575758),
 ('Tua Tagovailoa', 5.757575757575758),
 ('Justin Herbert', 5.748792270531402),
 ('Jameis Winston', 5.554515050167224),
 ('Baker Mayfield', 5.517512274959083),
 ('Caleb Williams', 5.465724751439037),
 ('Kyler Murray', 5.2323955669224205),
 ('Trevor Lawrence', 5.027641245177316),
 ('Bo Nix', 4.82398303450935),
 ('Gardner Minshew', 4.798920377867747),
 ('Daniel Jones', 4.194432169686257),
 ('Bryce Young', 3.548872180451128)]